# Backward chaining and goal trees 

# Going Backwards

Backward chaining is the opposite of forward chaining: we run a production rule system in reverse. We start with a conclusion (the hypothesis), and then we find which rules, when fired, would yield that hypothesis. Then, we test those rules' antecedents to figure out how we can successfully match them against data we have or other rules' consequents.
The Backward Chaining Process

Here's the general theory behind backward chaining:

    Given a hypothesis, you want to see what rules can produce it, by matching the consequents of those rules against your hypothesis. All of the rules' consequents that match are possible options, so you'll take the corresponding antecedents and group them together in an OR node: it is sufficient for any one matching rule to fire and yield the hypothesis.
    The hypothesis itself should be added as a (leaf) node to the OR node, because existence of the hypothesis in the data is sufficient to conclude the hypothesis.
    If a consequent matches, keep track of the variables that are bound. Look up the antecedent of that rule, and instantiate those same variables in the antecedent (that is, replace the variables with their values). This instantiated antecedent is a new hypothesis.
    The antecedent may have AND or OR expressions. This means that the goal tree for the antecedent is already partially formed. But you need to check the leaves of that AND-OR tree, and recursively backward chain on them. 

A few things you should be aware of:

    The branches of the goal tree should be in order: the goal trees for earlier rules should appear before (to the left of) the goal trees for later rules. Intermediate nodes should appear before their expansions.
    The output should be fully simplified (you can use the simplify function).
    If two different rules tell you to check the same hypothesis, the goal tree for that hypothesis should be included both times, even though it's largely redundant. 

# Part 4: Backward Chaining

In this problem, we will do backward chaining by starting from a conclusion, and generating a goal tree of all of the statements we may need to test. The leaves of the goal tree will be sentences (strings) like 'opus swims', indicating atomic failure or success based on whether or not 'opus swims' is in our assertions list.

We'll run this backward chainer on the zookeeper system of rules, a simple set of production rules for classifying animals, which you will find in data.py. As an example, here is the goal tree generated for the hypothesis 'opus is a penguin':

OR(
  'opus is a penguin',
  AND(
    OR('opus is a bird', 'opus has feathers', AND('opus flies', 'opus lays eggs'))
    'opus does not fly',
    'opus swims',
    'opus has black and white color' ))

You will write a procedure, backchain_to_goal_tree(rules, hypothesis) (in "Part 4" of lab1.py), which outputs the goal tree containing the statements you would need to test to prove the hypothesis. Note that this function is supposed to be a general backchainer, so you should not hard-code anything that is specific to a particular rule set. The backchainer will be tested on rule sets other than zookeeper_rules.

The rules you work with will be limited in scope, because general-purpose backward chainers are difficult to write. In particular, for this problem, make the following assumptions:

    All variables that appear in a rule's antecedent also appear in its consequent (so there are no "unknown" variables in the antecedent). In other words, you will not need to do backtracking.
    All assertions are positive: no rules will have DELETE clauses or NOT expressions.
    Rule antecedents never have nested RuleExpression nodes. For example, something like (OR (AND x y) (AND z w)) will never appear within an antecedent, because that comprises an AND expression nested under an OR expression.
    Rule consequents always have just a single statement. 

Note that an antecedent can be a single hypothesis (a string) or a RuleExpression. 

 production.py API

The code in production.py has been written by 6.034 staff. It contains all of the infrastructure supporting IF/THEN rules and how they're processed. So that you don't have to trawl through our code trying to figure out how everything works, we use this section to provide you with a quick explanation of which functions you will find useful and how you should interact with them.

match(pattern, datum)
    This attempts to assign values to variables so that pattern and datum are the same. You can match(leaf_a, leaf_b), which returns either None if leaf_a didn't match leaf_b, or a set of bindings if it did (even empty bindings: {}). 

    Examples:

        match("(?x) is a (?y)", "John is a student") => { x: "John", y: "student" } 
        match("foo", "bar") => None 
        match("foo", "foo") => {} 

    Both arguments to match must be strings. 

    Note: {} and None are both False-like values in Python, so you should make sure to explicitly check if match's return value is None (i.e. don't use idioms like "if match(a, b):"). If match returns {}, that means that the statements match but there are no variables that need to be bound, whereas if match returns None, that means it's not possible for the statements to match at all! 

populate(exp, bindings)
    Given an expression with variables in it, look up the values of those variables in bindings and replace the variables with their values. You can use the bindings from match(leaf_a, leaf_b) with populate(leaf, bindings), which will fill in any free variables using the bindings. Note that the expression input to populate may either be a string or a more complicated tree. 

    Examples:

        populate("(?x) is a (?y)", { x: "John", y: "student" }) => "John is a student" 
        populate(AND("(?x) is a (?y)", "(?x) loves (?z)"), { x: "John", y: "student" }) => AND("John is a student", "John loves (?z)") 

rule.antecedent()
    Returns the IF part of a rule, which is either a leaf or a RuleExpression. 

    Recall that RuleExpression objects act like lists, so you can iterate over them. If you need to know to what class an antecedent belongs, you may find the isinstance function to be helpful. For example, isinstance(my_antecedent, OR) returns True if my_antecedent is an OR object, otherwise it returns False. 

rule.consequent()
    Returns the THEN part of a rule, which is always a single statement for the purposes of this lab. (Note that rule.consequent() does not return a THEN object; it returns the datum enclosed by the THEN object.) 

In [8]:
from production import IF, AND, OR, NOT, THEN, DELETE, forward_chain, pretty_goal_tree
from production import PASS, FAIL, match, populate, simplify, variables
from backchain import backchain_to_goal_tree
from data import *
import pprint

pp = pprint.PrettyPrinter(indent=1)
pprint = pp.pprint

ModuleNotFoundError: No module named 'zookeeper'

In [5]:
def backchain_to_goal_tree(rules, hypothesis):
    """
    Takes a hypothesis (string) and a list of rules (list
    of IF objects), returning an AND/OR tree representing the
    backchain of possible statements we may need to test
    to determine if this hypothesis is reachable or not.

    This method should return an AND/OR tree, that is, an
    AND or OR object, whose constituents are the subgoals that
    need to be tested. The leaves of this tree should be strings
    (possibly with unbound variables), *not* AND or OR objects.
    Make sure to use simplify(...) to flatten trees where appropriate.
    """
    #find a rule which has the hypothesis as the conclusion and testing its premises
    # Rule Z14:
    IF( AND( '(?x) is a bird',   #in rule Z3       
             '(?x) does not fly', #opposite of rule Z4
             '(?x) swims',
             '(?x) has black and white color' ),
        THEN( '(?x) is a penguin' ))
       # penguin = opus?
    #OR('opus is a penguin', AND(
    #OR('opus is a bird', 'opus has feathers'), 
    #AND('opus flies', 'opus lays eggs'))
    #'opus does not fly',
    #'opus swims',
    #'opus has black and white color' ))
       #Rule Z14: if 

    results = [hypothesis]
    for rule in rules:
        consequent = rule.consequent()
        for expr in consequent:
            bindings = match(expr, hypothesis)
            if bindings or expr == hypothesis:
                antecedent = rule.antecedent()
                if isinstance(antecedent, str):
                    new_hypothesis = populate(antecedent, bindings)
                    results.append(backchain_to_goal_tree(rules, new_hypothesis))
                    results.append(new_hypothesis)
                else:
                    statements = [populate(ante_expr, bindings) for ante_expr in antecedent]
                    new_results = []
                    for statement in statements:
                        new_results.append(backchain_to_goal_tree(rules, statement))
                    results.append(create_statement(new_results, antecedent))
    return simplify(OR(results))

def create_statement(statements, rule):
    if isinstance(rule, AND):
        return AND(statements)
    elif isinstance(rule, OR):
        return OR(statements)


# Uncomment this to test out your backward chainer:
pretty_goal_tree(backchain_to_goal_tree(zookeeper_rules, 'opus is a penguin'))




error: missing ), unterminated subpattern at position 0